### Solving Sparse Regression using `NExOS.jl`

The sparse regression problem (also known as regressor selection problem) is concerned with approximating a vector $b\in\mathbf{R}^{m}$ with a linear combination of at most $k$ columns of a matrix $A\in\mathbf{R}^{m\times d}$ with bounded coefficients. The problem can be written as the following optimization
problem
$$
\begin{equation}
\begin{array}{ll}
\textrm{minimize} & \|Ax-b\|_{2}^{2}+\frac{\beta}{2}\|x\|^{2}\\
\textrm{subject to} & \mathbf{card}(x)\leq k\\
 & \|x\|_{\infty}\leq M,
\end{array}
\end{equation}
$$
where $x\in\mathbf{R}^{d}$ is the decision variable, and $A\in\mathbf{R}^{m\times d},b\in\mathbf{R}^{m},$ and $M>0$ are problem data.

First, load the packages.

In [1]:
using Random, NExOS, ProximalOperators

Let us generate some random data for this problem.

In [2]:
m = 25
n = 50
A = randn(m,n)
A = randn(m,n)
b = randn(m)
M = 100
k = convert(Int64, round(m/3))
beta = 10^-10

1.0000000000000006e-10

Create the problem instance in `NExOS`.

In [3]:
C = SparseSet(M, k) # Create the set
f = LeastSquares(A, b, iterative = true) # Create the function
setting = NExOS.Setting(μ_max = 2, μ_min = 1e-8, μ_mult_fact = 0.85, verbose = false, freq = 250, γ_updt_rule = :adaptive, β = beta) # setting
z0 = zeros(n) # create an initial point
problem = NExOS.Problem(f, C, setting.β, z0) # problem instance

Problem{ProximalOperators.LeastSquaresIterative{1,Float64,Float64,Array{Float64,2},Array{Float64,1},ProximalOperators.AAc},SparseSet{Int64,Int64},Float64,Array{Float64,1}}(description : Least squares penalty
domain      : n/a
expression  : n/a
parameters  : n/a, SparseSet{Int64,Int64}(100, 8), 1.0000000000000006e-10, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

Time to solve the problem.

In [4]:
state_final = NExOS.solve!(problem, setting)

State{Array{Float64,1},Int64,Float64}([2.3604068371270658e-9, -1.8614022509121833e-8, 1.4283929504262508e-8, -0.16944938189544959, 1.607411136746223e-8, 1.9905400634770604e-8, 0.29495341217575866, 0.20254672745957286, 3.219912492554042e-8, 2.295088435599488e-8  …  -0.17321307976429576, -4.257769720060745e-8, 0.3490200698603441, -4.066531833939707e-9, -1.8881561945230612e-9, 1.6860244710903703e-8, -6.390798535020245e-9, -7.4146058898080014e-9, 2.3471740858500734e-8, 2.9259207732582412e-8], [2.503142602875358e-9, -1.9739343790018626e-8, 1.5147520155775593e-8, -0.1694493825596602, 1.7045900328179816e-8, 2.1108862617962955e-8, 0.2949534115875845, 0.2025467262194669, 3.414578090974902e-8, 2.4338410440704434e-8  …  -0.17321307916293596, -4.5151862513441285e-8, 0.3490200698298937, -4.312348720089836e-9, -2.0023296492550167e-9, 1.7879582823859328e-8, -6.777157090410619e-9, -7.86286864718721e-9, 2.4890840888034824e-8, 3.1028160171478726e-8], [-2.347730063129355e-8, 1.8513760949107962e-7, -1.420

Let us take a look at the quality of the solution.

In [5]:
log10(state_final.fxd_pnt_gap) <= -4 # if the fixed point gap is less than 10^-4 (to determin if the algorithm has converged)

true

In [6]:
log10(state_final.fsblt_gap) <= -4 # this is to test if the found solution by NExOS is locally optimal

true

In [7]:
f(state_final.x) # this gives the objective value of the solution found by NExOS

2.6773713078392